# Reading an Processind dataset

In [2]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, Float, String
import unicodedata
import re

def normalize_string(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s.strip().lower()) if unicodedata.category(c) != 'Mn')

df = pd.read_csv('dataset.csv')

df = df[["artists", "album_name", "track_name","popularity", "duration_ms", "explicit","loudness"]]
df.drop_duplicates(inplace=True)
df["artists"] = df["artists"].apply(lambda x: normalize_string(str(x)))
df["album_name"] = df["album_name"].apply(lambda x: normalize_string(str(x)))
df["track_name"] = df["track_name"].apply(lambda x: normalize_string(str(x)))




# MySQL

In [5]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="alex1234"
)

mycursor = mydb.cursor()
mycursor.execute('DROP DATABASE IF EXISTS spotify')
mycursor.execute("CREATE DATABASE spotify")
mydb.close()

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="alex1234",
  database="spotify"
)

mycursor = mydb.cursor()

mycursor.execute('DROP Table IF EXISTS artists')
mycursor.execute('DROP Table IF EXISTS albums')
mycursor.execute('DROP Table IF EXISTS tracks')
mycursor.execute('DROP Table IF EXISTS track_artists')
mycursor.execute("CREATE TABLE artists (id INT PRIMARY KEY, name VARCHAR(255) UNIQUE)")
mycursor.execute("CREATE TABLE albums (id INT PRIMARY KEY, name VARCHAR(255), artist_id INT, FOREIGN KEY (artist_id) REFERENCES artists(id), UNIQUE (name, artist_id))")
mycursor.execute("CREATE TABLE tracks (id INT PRIMARY KEY, name VARCHAR(1000), album_id INT, popularity INT,\
                  duration_ms INT, explicit BOOL, loudness FLOAT ,FOREIGN KEY (album_id) REFERENCES albums(id))")
mycursor.execute("CREATE TABLE track_artists (id INT AUTO_INCREMENT PRIMARY KEY, track_id INT, \
                 artist_id INT, FOREIGN KEY (track_id) REFERENCES tracks(id), FOREIGN KEY (artist_id) REFERENCES artists(id))")


add_record_artists = ("INSERT INTO artists (id,name) VALUES (%s,%s)")
add_record_albums = ("INSERT INTO albums (id,name,artist_id) VALUES (%s,%s, %s)")
add_record_tracks = ("INSERT INTO tracks (id,name, album_id,popularity,duration_ms, explicit,loudness) VALUES (%s,%s, %s, %s, %s, %s, %s)")
add_record_track_artists = ("INSERT INTO track_artists (track_id, artist_id) VALUES (%s, %s)")

artists_values = []
albums_values = []
tracks_values = []
track_artists_values = []
artists_id = dict()# name:id
albums_id = dict()# (album_name, artist_id):id
curr_artists_id = 0
curr_albums_id = 0
curr_track_id = 0

for [artists_str,album_name,track_name,popularity,duration_ms, explicit, loudness] in df.values.tolist():
    track_artists_ids = []
    album_id = None
    track_id = None
    artists = artists_str.split(";")
    if not re.match('^[a-zA-Z0-9 !%&-.,]+$', album_name): continue
    for artist_name in artists:
        if not re.match('^[a-zA-Z0-9 !%&-.,]+$', artist_name): continue
        try:
            if artist_name not in artists_id:
                artists_values.append((curr_artists_id, artist_name))
                artists_id[artist_name] = curr_artists_id
                curr_artists_id += 1
            track_artists_ids.append( artists_id[artist_name])
        except mysql.connector.errors.IntegrityError:
            continue

    if len(track_artists_ids) == 0: continue
    main_artist_id = track_artists_ids[0]
    album_key = (album_name,main_artist_id)
    try:
        if album_key not in albums_id:
            albums_values.append((curr_albums_id, album_name, track_artists_ids[0]))
            albums_id[album_key] = curr_albums_id
            curr_albums_id += 1
        album_id = albums_id[album_key]
    except mysql.connector.errors.IntegrityError:
        continue
    
    try:
        tracks_values.append((curr_track_id, track_name, album_id, popularity, duration_ms, explicit, loudness))
        track_id = curr_track_id
        curr_track_id += 1
    except: 
        continue

    for artist_id in track_artists_ids:
        track_artists_values.append((track_id, artist_id))


try:
    mycursor.executemany(add_record_artists, artists_values)
    mycursor.executemany(add_record_albums, albums_values)
    mycursor.executemany(add_record_tracks, tracks_values)
    mycursor.executemany(add_record_track_artists, track_artists_values)
    mydb.commit()
except:
    mydb.close()

mydb.close()
        

# MongoDB

In [3]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
client.drop_database('spotify')
db = client.spotify

artists = db.artists
albums = db.albums
tracks = db.tracks

artists_insert = list()
albums_insert = list()
track_insert = list()

tracks_in_album = dict() # (artist_name, album_name): [tracks]

artists_seen = dict()# artist unique key is name
artist_id = 1
albums_seen = set() # album unique key is (artist_name, album_name)
track_seen = set() # track unique key is ([artist_names], album_name, track_name)


for [artists_str,album_name,track_name,popularity,duration_ms, explicit, loudness] in df.values.tolist():
    track_artists = []
    if not re.match('^[a-zA-Z0-9 !%&-.,]+$', album_name): continue
    try:
        artists_list = artists_str.split(";")
    except:
        continue
    main_artist = artists_list[0]
    for artist in artists_list:
        if not re.match('^[a-zA-Z0-9 !%&-.,]+$', artist): continue
        artist_obj = {"_id":artist_id,"name": artist}

        if artist not in artists_seen:
            artists_insert.append(artist_obj)
            artists_seen[artist] = artist_id
            artist_id += 1
        else:
            artist_obj["_id"] = artists_seen[artist]
        track_artists.append(artist_obj)
    
    if len(track_artists) == 0: continue

    track_obj = {"name": track_name, "artists":track_artists ,"popularity": popularity,
                  "duration_ms": duration_ms, "explicit": explicit, "loudness": loudness}


    track_key = (tuple(map(lambda o: o["name"],track_artists)), album_name, track_name)
    if track_key not in track_seen:
        track_insert.append(track_obj)
        track_seen.add(track_key)

    track_album_key = (main_artist, album_name)
    tracks_in_album[track_album_key] = [track_obj] if track_album_key not in tracks_in_album else tracks_in_album[track_album_key] + [track_obj]

for [artists_str,album_name,track_name,popularity,duration_ms, explicit, loudness] in df.values.tolist():
    if not re.match('^[a-zA-Z0-9 !%&-.,]+$', album_name): continue
    try:
        artists_list = artists_str.split(";")
    except:
        continue
    if not re.match('^[a-zA-Z0-9 !%&-.,]+$', artists_list[0]): continue
    artist_album_obj =  {"name": artists_list[0]}

    track_album_key = (artist_album_obj["name"], album_name)
    album_obj = {"name": album_name, "artist_id": artists_seen[artists_list[0]], "tracks": tracks_in_album[track_album_key]}

    album_key = (artist_album_obj["name"], album_name)
    if album_key not in albums_seen:
        albums_insert.append(album_obj)
        albums_seen.add(album_key)




artists.insert_many( artists_insert)
tracks.insert_many( track_insert)
albums.insert_many(albums_insert)